In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**1. DATA LOAD**

In [ ]:
train = pd.read_csv('/kaggle/input/sf-crime/train.csv.zip')
train.head()

In [ ]:
test = pd.read_csv('/kaggle/input/sf-crime/test.csv.zip')
test.head()
# train에서 Category, Descript, Resolution 변수가 없음.

In [ ]:
sub = pd.read_csv('/kaggle/input/sf-crime/sampleSubmission.csv.zip')
sub.head()
# class 39개

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
sub.shape

In [ ]:
train.info() #.dtypes

**2. DATA EDA**

In [ ]:
train["Dates"] = pd.to_datetime(train["Dates"])
train["YEAR"] = train["Dates"].dt.year
train["MONTH"] = train["Dates"].dt.month
train["DAY"] = train["Dates"].dt.day
train["HOUR"] = train["Dates"].dt.hour

test["Dates"] = pd.to_datetime(test["Dates"])
test["YEAR"] = test["Dates"].dt.year
test["MONTH"] = test["Dates"].dt.month
test["DAY"] = test["Dates"].dt.day
test["HOUR"] = test["Dates"].dt.hour

In [ ]:
train.info()
#train.dtypes
# 변수 타입이 object인 것은  수치/범주형으로 변경

In [ ]:
train = train.astype({'Category' : 'category', 
                      'DayOfWeek' : 'string', 
                      'PdDistrict': 'string',
                      'Resolution' : 'string',
                      'Address' : 'string'})
train.dtypes

In [ ]:
test = test.astype({'DayOfWeek' : 'string', 
                    'PdDistrict': 'string',
                    'Address' : 'string'})
test.dtypes

In [ ]:
# target variable
import matplotlib.pyplot as plt
# import seaborn as sns

TARGET = train['Category']
DF_ = TARGET.value_counts()
len_x = np.arange(len(DF_))

plt.figure(figsize = (20, 12))
plt.bar(len_x, DF_)
plt.title("distribution of target variable(categorical)")
plt.ylabel('count')
plt.xlabel('Category')
plt.xticks(range(0, len(DF_.index)), DF_.index, rotation = "vertical")

# TRIAN_SUM_BY_CATE = train.groupby('Category')
# TRIAN_SUM_BY_CATE.count()
# plt.bar(3, DF_)

In [ ]:
DF_

**2 random forest**

In [ ]:
train_x = train[['DayOfWeek', 'PdDistrict', 'Address', 
                 'X', 'Y', 'YEAR', 'MONTH', 'DAY', 'HOUR']]
train_y = train[['Category']]

test_x = test[['DayOfWeek', 'PdDistrict', 'Address', 
                 'X', 'Y', 'YEAR', 'MONTH', 'DAY', 'HOUR']]



train_x.head()

In [ ]:
train_x.info()

In [ ]:
train_y.head()

In [ ]:
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

le = preprocessing.LabelEncoder()

train_x["DayOfWeek"] = le.fit_transform(train_x["DayOfWeek"])
train_x["PdDistrict"] = le.fit_transform(train_x["PdDistrict"])
train_x["Address"] = le.fit_transform(train_x["Address"])

test_x["DayOfWeek"] = le.fit_transform(test_x["DayOfWeek"])
test_x["PdDistrict"] = le.fit_transform(test_x["PdDistrict"])
test_x["Address"] = le.fit_transform(test_x["Address"])


In [ ]:
clf = RandomForestClassifier(n_estimators = 20, 
                             max_depth = 5, random_state = 0)
rf1 = clf.fit(train_x, train_y)

In [ ]:
predictions = rf1.predict_proba(test_x)

In [ ]:
predictions

In [ ]:
predictions.shape

In [ ]:
np.linspace(0, 38, 39, dtype='int16')

In [ ]:
sub_length = le.inverse_transform(np.linspace(0, 38, 39, dtype='int16'))
sub_length

In [ ]:
train_y['Category'].unique()

In [ ]:
sub.head()

In [ ]:
len(predictions[0])

In [ ]:
sub.iloc[:,1:] = predictions

In [ ]:
sub.head()

In [ ]:
sub.to_csv('sub1.csv', index = False)